In [15]:
import os
import numpy as np
import pandas as pd
from custom import utils
from collections import Counter
import inspect
import scipy.io

### Function for dealing with the handedness covariate
This function performs categorical imputation. 
    This decision is made based on the fact that ~90 out of 1000+ 
    handed values are missing. That small percentage is imputed 
    with the most frequent handedness score "Right". There's also
    a very small percentage of non right non left values that 
    are a mix of "Both", "Mixed", "Either", "Ambidextrous", these
    are all imputed to just "Both"

In [54]:
def most_handed(data):
    """data: pd.DataFrame"""
    counts = Counter(data)
    most = max(counts.items())[0]
    data = data.copy().fillna(0)
    data[data == 0] = 'Right'
    both = ['Both','Mixed','Either','Ambidextrous']
    for hand in both:
        data[data == hand] = 'Both'
    return data

def mean_center_n_scale(data):
    data_mean_zero = data - data.mean(0)
    data_scaled = data_mean_zero / data_mean_zero.std(0)
    return data_scaled

## Preprocessing function
Inputs are paths to data, sMRI features and SNPs respectively. "bcvar" is a list of covariates: ['SEX', 'AGE_MRI', 'EstimatedTotalIntraCranialVol', 'STUDY']. "brain_cols" is a numpy array of feature names to subset the sMRI data. 
First thing that happens is I get a boolen array of where the brain data contains only Controls or Schizophrenics in the Group feature. Both datasets are subsetted by this boolean array, row wise. It's important to note that the datasets loaded are already in the same order rowwise. Then I create 2 sets of covariate matrices and concatenate them into one. The first set contains AGE, SEX, and ICV(EstimatedTotalIntraCranialVol), the second one is a one hot encoded matrix of handedness, the third is a one hot encoded matrix of site(i.e, study). The first step of the analysis does not use the response variable but I safe it to apply stratifiedKFold cross-validation. Two dictionaries are returned, one contains keys and numpy arrays. The name of the keys are required inputs for the first part of the analysis. The dictionary containing the column headers is not required but is saved for later uses. This function also applies mean centering and scaling

In [55]:
def preprocess(brain_path, snp_path, bcvar, brain_cols):
    """brain_path: String, snp_path: String, bcvar: list, 
    brain_cols: list or np.ndarray. 
    """
    # load data
    brain_data = pd.read_hdf(brain_path)
    snp_data = pd.read_hdf(snp_path)
    # get the group status
    gr = brain_data.GROUP.values
    cnt_scz = np.logical_or(gr == 'Control', gr == 'Schizophrenia')
    # subset by indexes cnt_scz
    brain_data = brain_data.iloc[cnt_scz, :]
    snp_data = snp_data.iloc[cnt_scz, :]
    # create set of covariates
    icv = 'EstimatedTotalIntraCranialVol'
    cov_set1 = pd.DataFrame(
        data=np.hstack((snp_data.SEX.values[:, None],
                        brain_data.AGE_MRI.values[:, None],
                        brain_data[icv].values[:, None])),
        columns=['SEX','AGE','EstimatedTotalIntraCranialVol'])
    cov_set1 = cov_set1.fillna(0)
    cov_set1[cov_set1.AGE == 0] = cov_set1.AGE.mean()
    #cov_site = utils.make_non_singular(utils.encoder(brain_data.STUDY.values))
    #cov_site_cols = ['site{}'.format(i) for i in range(cov_site.shape[1])]
    #cov_site = pd.DataFrame(data=cov_site, columns=cov_site_cols)
    cov_hand = utils.encoder(most_handed(brain_data.HANDED))
    cov_hand_cols = ['handed{}'.format(i) for i in range(cov_hand.shape[1])]
    cov_hand = pd.DataFrame(data=cov_hand, columns=cov_hand_cols)
    cvars = pd.concat([cov_set1, cov_hand], axis=1)
    y = np.array([0 if i == 'Control' else 1 for i in brain_data.GROUP.values])
    return {'Z': mean_center_n_scale(cvars.values), 
            'I': mean_center_n_scale(brain_data[brain_cols].values), 
            'G': mean_center_n_scale(snp_data.iloc[:, 1:-5].values),
            'colnames': snp_data.iloc[:, 1:-5].columns.values,
            'y':y}, {'Z_cols':cvars.columns.values,
                     'I_cols':brain_cols,
                     'G_cols': snp_data.iloc[:, 1:-5].columns.values}

# Helper functions
These two functions assist in the analysis. save_preprocessed saves the data to disk that are outputed from the preprocess function above. The paths to where the files are written on disk are returned. This is because I'll be using nipype so to make life easier data isn't passed when interfacing with nipype nodes - just the path to where the data lives. I then load the data using the paths. The cv_maker function creates k-fold stratified cross validation indices and saves them. These indices are used in the matlab script to load the correct subsets of data. 

In [56]:
def save_preprocessed(preproc_data_dict, preproc_data_dict_col, save_path, dn, cn):
    """preproc_data_dict: dictionary object returned from
    the preprocessing function, (the first - zeroth value of the return) 
    preproc_data_dict_col: dintionary object returned from
    the preprocessing function, (the second - first value of the return)
    save_path: string - base path for saving the dictionaries
    dn: string - name for saving the data dictionary 
    cn: string - name for saving the column header dictionary
    """
    save_dict = os.path.join(save_path, dn)
    save_cols = os.path.join(save_path, cn)
    scipy.io.savemat(save_dict, mdict=preproc_data_dict)
    utils.save_pickle(save_cols, preproc_data_dict_col)
    return save_dict, save_cols

def cv_maker(data_path, save_path):
    import scipy.io
    from sklearn.model_selection import StratifiedKFold
    X = scipy.io.loadmat(data_path)['I']
    y = scipy.io.loadmat(data_path)['y'][0]
    cv = StratifiedKFold(n_splits=5, random_state=1)
    train_idx, test_idx = {}, {}
    for idx, (train, test) in enumerate(cv.split(X, y)):
        train_idx['train_{}'.format(idx + 1)] = train + 1
        test_idx['test_{}'.format(idx + 1)] = test + 1
    scipy.io.savemat(save_path, mdict={"train":train_idx, "test":test_idx})
    return save_path

# Create the input data, save the CV indices
The step below runs the functions I've made above

In [57]:
headers_dir = "/storage/gablab001/data/genus/fs_cog/pred_diag/column_headers"
brain_cols = np.genfromtxt(os.path.join(headers_dir, "XB"), dtype=str)
brain_path = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/brain_N1547_P5927_matched.hdf5py"
snp_path = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/genomic_N1547_P100006_matched.hdf5py"
bcv = ['SEX', 'AGE_MRI', 'EstimatedTotalIntraCranialVol', 'STUDY']
all_data, all_cols = preprocess(brain_path, snp_path, bcv, brain_cols)
path_for_save = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets"
for_cv, _ = save_preprocessed(all_data, all_cols, path_for_save, "brain_gene.mat","brain_gene_cols.pkl")
cv_path = cv_maker(for_cv, os.path.join(path_for_save, "cv_idx.mat"))

In [52]:
from nipype import Function, Node, Workflow, IdentityInterface

# Bayesian analysis - "step 1"
Below I create the workflow that I use with nipype, create the nipype wrapper nodes to wrap functions that will go into the nipype graph, and then submit the jobs. Due to the nature of the analysis we are parallelizing over the feature space in the sMRI data. That is - one job per feature, on top of that we are parallelizing the cross validation step. In total this means there are (170*10) jobs that need to be submitted. For a single user in my experience that's too many jobs for the Openmind cluster so I limit the amount of jobs that can be submitted at a time. 

In [59]:
CV_maker = Node(interface=Function(
    input_names = ['data_path', 'save_path'],
    output_names = ['save_path'],
    function = cv_maker
), name = 'CV_maker')

#CV_maker.inputs.data_path = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/brain_gene.mat"
#CV_maker.inputs.save_path = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/cv_idx.mat"


wf = Workflow(name='brain_bcv')
wf.base_dir = "/om/scratch/Tue/ysa"

Iternode = Node(IdentityInterface(fields=['col_idx', 'cv_idx']), name = 'Iternode')
Iternode.iterables = [('col_idx', np.arange(170) + 1), ('cv_idx', np.arange(5) + 1)]

def run_bayes(in_file, cv_file, cv_idx, col_idx, out_file):
    import cPickle as pickle
    import numpy as np
    import os
    import nipype.interfaces.matlab as Matlab
    def outnames(col, out):
        return os.path.join(out, '{}.mat'.format(col))
    headers_dir = "/storage/gablab001/data/genus/fs_cog/pred_diag/column_headers"
    col_names = np.genfromtxt(os.path.join(headers_dir, "XB"), dtype=str)
    col_save_name = col_names[col_idx - 1] + "_{}_{}_BF".format(cv_idx, col_idx)
    with open("/storage/gablab001/data/genus/GIT/genus/bayes/matlab/bayes_reg.m", "r") as src:
        script = src.read().replace("\n", "")
    mat_file = outnames(in_file[:-4]+'_'+col_save_name, out_file)
    matlab = Matlab.MatlabCommand()
    matlab.inputs.script = script.format(in_file, cv_file, cv_idx, col_idx, mat_file)
    res = matlab.run()
    return mat_file

Run_bayes = Node(interface=Function(
    input_names = ['in_file', 'cv_file','cv_idx',
                   'col_idx','out_file'],
    output_names = ['mat_file'],
    function = run_bayes
), name='Run_bayes')

Run_bayes.inputs.in_file = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/brain_gene.mat"
Run_bayes.inputs.cv_file = "/storage/gablab001/data/genus/GIT/genus/bayes/data_sets/cv_idx.mat"
Run_bayes.inputs.out_file = "/storage/gablab001/data/genus/GIT/genus/bayes/results/bayes_factor"

wf.connect(Iternode, 'cv_idx', Run_bayes, 'cv_idx')
wf.connect(Iternode, 'col_idx', Run_bayes, 'col_idx')
wf.run(plugin='SLURM', plugin_args={'sbatch_args':'--mem=4G -t 23:00:00', 'max_jobs': 150})

170725-14:51:22,760 workflow INFO:
	 Workflow brain_bcv settings: ['check', 'execution', 'logging']
170725-14:51:24,657 workflow INFO:
	 Running in parallel.
170725-14:51:24,678 workflow INFO:
	 Pending[0] Submitting[150] jobs Slots[150]
170725-14:51:24,681 workflow INFO:
	 Submitting: Run_bayes.a454 ID: 0
170725-14:51:24,767 workflow INFO:
	 Finished submitting: Run_bayes.a454 ID: 0
170725-14:51:24,768 workflow INFO:
	 Submitting: Run_bayes.a339 ID: 1
170725-14:51:24,860 workflow INFO:
	 Finished submitting: Run_bayes.a339 ID: 1
170725-14:51:24,862 workflow INFO:
	 Submitting: Run_bayes.a455 ID: 2
170725-14:51:24,954 workflow INFO:
	 Finished submitting: Run_bayes.a455 ID: 2
170725-14:51:24,955 workflow INFO:
	 Submitting: Run_bayes.a340 ID: 3
170725-14:51:25,46 workflow INFO:
	 Finished submitting: Run_bayes.a340 ID: 3
170725-14:51:25,48 workflow INFO:
	 Submitting: Run_bayes.a521 ID: 4
170725-14:51:25,136 workflow INFO:
	 Finished submitting: Run_bayes.a521 ID: 4
170725-14:51:25,137

170725-14:51:29,733 workflow INFO:
	 Submitting: Run_bayes.a812 ID: 53
170725-14:51:29,826 workflow INFO:
	 Finished submitting: Run_bayes.a812 ID: 53
170725-14:51:29,828 workflow INFO:
	 Submitting: Run_bayes.a770 ID: 54
170725-14:51:29,917 workflow INFO:
	 Finished submitting: Run_bayes.a770 ID: 54
170725-14:51:29,918 workflow INFO:
	 Submitting: Run_bayes.a619 ID: 55
170725-14:51:30,9 workflow INFO:
	 Finished submitting: Run_bayes.a619 ID: 55
170725-14:51:30,10 workflow INFO:
	 Submitting: Run_bayes.a198 ID: 56
170725-14:51:30,103 workflow INFO:
	 Finished submitting: Run_bayes.a198 ID: 56
170725-14:51:30,104 workflow INFO:
	 Submitting: Run_bayes.a631 ID: 57
170725-14:51:30,194 workflow INFO:
	 Finished submitting: Run_bayes.a631 ID: 57
170725-14:51:30,196 workflow INFO:
	 Submitting: Run_bayes.a200 ID: 58
170725-14:51:30,291 workflow INFO:
	 Finished submitting: Run_bayes.a200 ID: 58
170725-14:51:30,293 workflow INFO:
	 Submitting: Run_bayes.a536 ID: 59
170725-14:51:30,387 workfl

170725-14:51:35,100 workflow INFO:
	 Finished submitting: Run_bayes.a402 ID: 107
170725-14:51:35,102 workflow INFO:
	 Submitting: Run_bayes.a231 ID: 108
170725-14:51:35,192 workflow INFO:
	 Finished submitting: Run_bayes.a231 ID: 108
170725-14:51:35,193 workflow INFO:
	 Submitting: Run_bayes.a330 ID: 109
170725-14:51:35,279 workflow INFO:
	 Finished submitting: Run_bayes.a330 ID: 109
170725-14:51:35,280 workflow INFO:
	 Submitting: Run_bayes.a271 ID: 110
170725-14:51:35,370 workflow INFO:
	 Finished submitting: Run_bayes.a271 ID: 110
170725-14:51:35,371 workflow INFO:
	 Submitting: Run_bayes.a405 ID: 111
170725-14:51:35,461 workflow INFO:
	 Finished submitting: Run_bayes.a405 ID: 111
170725-14:51:35,462 workflow INFO:
	 Submitting: Run_bayes.a390 ID: 112
170725-14:51:35,552 workflow INFO:
	 Finished submitting: Run_bayes.a390 ID: 112
170725-14:51:35,554 workflow INFO:
	 Submitting: Run_bayes.a426 ID: 113
170725-14:51:35,647 workflow INFO:
	 Finished submitting: Run_bayes.a426 ID: 113
1

170725-14:52:30,996 workflow INFO:
	 Pending[149] Submitting[1] jobs Slots[1]
170725-14:52:30,998 workflow INFO:
	 Submitting: Run_bayes.a539 ID: 150
170725-14:52:31,154 workflow INFO:
	 Finished submitting: Run_bayes.a539 ID: 150
170725-14:54:30,274 workflow ERROR:
	 [u'Node Run_bayes.a631 failed to run on host node002.cm.cluster.']
170725-14:54:30,276 workflow INFO:
	 Saving crash info to /storage/gablab001/data/genus/GIT/genus/bayes/notebooks/crash-20170725-145430-ysa-Run_bayes.a631-dd81f0cf-46fe-49e7-9c07-53b5ccd568d8.pklz
170725-14:54:30,278 workflow INFO:
	 Traceback (most recent call last):
  File "/om/scratch/Tue/ysa/brain_bcv/batch/pyscript_20170725_145130_brain_bcv_Run_bayes.a631.py", line 32, in <module>
    result = info['node'].run(updatehash=info['updatehash'])
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packag

170725-14:54:32,324 workflow ERROR:
	 [u'Node Run_bayes.a085 failed to run on host node002.cm.cluster.']
170725-14:54:32,326 workflow INFO:
	 Saving crash info to /storage/gablab001/data/genus/GIT/genus/bayes/notebooks/crash-20170725-145432-ysa-Run_bayes.a085-54f6719b-e76f-401a-868f-b67fdbdabf34.pklz
170725-14:54:32,329 workflow INFO:
	 Traceback (most recent call last):
  File "/om/scratch/Tue/ysa/brain_bcv/batch/pyscript_20170725_145132_brain_bcv_Run_bayes.a085.py", line 32, in <module>
    result = info['node'].run(updatehash=info['updatehash'])
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 482, in _run_interface
    self._result = self._run_command(execute)
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 613, in _ru

...

170725-15:55:35,302 workflow INFO:
	 Saving crash info to /storage/gablab001/data/genus/GIT/genus/bayes/notebooks/crash-20170725-155535-ysa-Run_bayes.a081-0fd0fc80-9092-491b-bdc8-8311d8ab5633.pklz
170725-15:55:35,306 workflow INFO:
	 Traceback (most recent call last):
  File "/om/scratch/Tue/ysa/brain_bcv/batch/pyscript_20170725_155307_brain_bcv_Run_bayes.a081.py", line 32, in <module>
    result = info['node'].run(updatehash=info['updatehash'])
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 482, in _run_interface
    self._result = self._run_command(execute)
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 613, in _run_command
    result = self._interface.run()
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site

170725-15:55:35,881 workflow INFO:
	 Pending[149] Submitting[1] jobs Slots[1]
170725-15:55:35,883 workflow INFO:
	 Submitting: Run_bayes.a833 ID: 542
170725-15:55:35,978 workflow INFO:
	 Finished submitting: Run_bayes.a833 ID: 542
170725-15:55:47,785 workflow ERROR:
	 [u'Node Run_bayes.a720 failed to run on host node004.cm.cluster.']
170725-15:55:47,787 workflow INFO:
	 Saving crash info to /storage/gablab001/data/genus/GIT/genus/bayes/notebooks/crash-20170725-155547-ysa-Run_bayes.a720-dd3e4ee7-8803-4e72-8734-a7d4c1ab63d4.pklz
170725-15:55:47,791 workflow INFO:
	 Traceback (most recent call last):
  File "/om/scratch/Tue/ysa/brain_bcv/batch/pyscript_20170725_155320_brain_bcv_Run_bayes.a720.py", line 32, in <module>
    result = info['node'].run(updatehash=info['updatehash'])
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packag

170725-15:55:48,174 workflow ERROR:
	 [u'Node Run_bayes.a721 failed to run on host node028.cm.cluster.']
170725-15:55:48,176 workflow INFO:
	 Saving crash info to /storage/gablab001/data/genus/GIT/genus/bayes/notebooks/crash-20170725-155548-ysa-Run_bayes.a721-dd559968-21bc-4050-b18d-a74b49b45e8d.pklz
170725-15:55:48,179 workflow INFO:
	 Traceback (most recent call last):
  File "/om/scratch/Tue/ysa/brain_bcv/batch/pyscript_20170725_155435_brain_bcv_Run_bayes.a721.py", line 32, in <module>
    result = info['node'].run(updatehash=info['updatehash'])
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 482, in _run_interface
    self._result = self._run_command(execute)
  File "/om/user/ysa/miniconda3/envs/py27/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 613, in _ru

170725-15:55:48,403 workflow INFO:
	 Pending[148] Submitting[2] jobs Slots[2]
170725-15:55:48,405 workflow INFO:
	 Submitting: Run_bayes.a009 ID: 543
170725-15:55:48,494 workflow INFO:
	 Finished submitting: Run_bayes.a009 ID: 543
170725-15:55:48,496 workflow INFO:
	 Submitting: Run_bayes.a717 ID: 544
170725-15:55:48,585 workflow INFO:
	 Finished submitting: Run_bayes.a717 ID: 544
170725-15:55:59,324 workflow INFO:
	 [Job finished] jobname: Run_bayes.a538 jobid: 502


KeyboardInterrupt: 